# Learning private models with multiple teachers

Protocol:
1. Train teachers:
    - Devide training set into buckets (not overlapping)
    - Train a models (teacher) on each bucket
2. Train student:
    - Extract a share of the test set
    - Ensemble predictions from teachers: queries each teacher for predictions on the test set share
    - Aggregate teacher predictions to get student training labels using noising max: it
  adds Laplacian noise to label counts and returns the most frequent label
    - Train student with the aggregated label
    - Validate the student model on the remaining test data

http://www.cleverhans.io/privacy/2018/04/29/privacy-and-machine-learning.html
https://github.com/tensorflow/models/tree/master/research/differential_privacy/multiple_teachers

In [1]:
import tensorflow as tf

from pate import ld_mnist, train_teachers, train_student

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader,Dataset

/anaconda3/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

tf.flags.DEFINE_string('dataset', 'mnist', 'The name of the dataset to use')
tf.flags.DEFINE_integer('nb_labels', 10, 'Number of output classes')

tf.flags.DEFINE_string('data_dir','/tmp','Temporary storage')
tf.flags.DEFINE_string('train_dir','/tmp/train_dir',
                       'Where model ckpt are saved')

tf.flags.DEFINE_integer('max_steps', 3000, 'Number of training steps to run.')
tf.flags.DEFINE_integer('nb_teachers', 100, 'Teachers in the ensemble.')
tf.flags.DEFINE_integer('teacher_id', 0, 'ID of teacher being trained.')


tf.flags.DEFINE_string('teachers_dir','/tmp/train_dir',
                       'Directory where teachers checkpoints are stored.')

tf.flags.DEFINE_integer('stdnt_share', 1000,
                        'Student share (last index) of the test data')
tf.flags.DEFINE_integer('lap_scale', 10,
                        'Scale of the Laplacian noise added for privacy')
tf.flags.DEFINE_boolean('save_labels', False,
                        'Dump numpy arrays of labels and clean teacher votes')
tf.flags.DEFINE_boolean('deeper', False, 'Activate deeper CNN model')


FLAGS = tf.flags.FLAGS

In [3]:
train_data, train_labels, test_data, test_labels = ld_mnist()

# Reshape to have channel first
train_data = train_data.reshape(60000,1,28,28)
test_data = test_data.reshape(10000,1,28,28)

In [4]:
dataset, nb_teachers, teacher_id = FLAGS.dataset, FLAGS.nb_teachers,  FLAGS.teacher_id

In [5]:
class CNN_Model(nn.Module):
    def __init__(self, num_classes):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 5, stride = 1)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()
        self.avgpool1 = nn.AvgPool2d(2)
        self.conv2 = nn.Conv2d(16, 16, 5, stride = 1)
        self.batchnorm2 = nn.BatchNorm2d(16)
        self.relu2 = nn.ReLU()
        self.avgpool2 = nn.AvgPool2d(2)
        self.linear1 = nn.Linear(256, 100)
        self.batchnorm3 = nn.BatchNorm1d(100)
        self.relu3 = nn.ReLU()
        self.linear2 = nn.Linear(100, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.relu1(x)
        x = self.avgpool1(x)
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = self.relu2(x)
        x = self.avgpool2(x)
        x = x.view(-1, 256)
        x = self.linear1(x)
        x = self.batchnorm3(x)
        x = self.relu3(x)
        out = self.linear2(x)
        return out

In [6]:
model = CNN_Model(10)

## Train Teachers

In [7]:
ckpt_path = 'checkpoint/'

for teacher_id in range(nb_teachers):
    
    filename = str(dataset) + '_' + str(nb_teachers) + '_teachers_' + str(teacher_id) + '.pth'

    train_teachers(model, train_data, train_labels, test_data, test_labels, nb_teachers, teacher_id, filename)


Train teacher ID: 0


/anaconda3/envs/py27/lib/python2.7/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Train Accuracy: 415/600 (69%)
Train Accuracy: 549/600 (92%)
Train Accuracy: 569/600 (95%)
Train Accuracy: 584/600 (97%)
Train Accuracy: 594/600 (99%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 600/10000 (94%)

Train teacher ID: 1
Train Accuracy: 540/600 (90%)
Train Accuracy: 580/600 (97%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 600/10000 (95%)

Train teacher ID: 2
Train Accuracy: 569/600 (95%)
Train Accuracy: 578/600 (96%)
Train Accuracy: 592/600 (99%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train A

Test Accuracy: 600/10000 (98%)

Train teacher ID: 23
Train Accuracy: 583/600 (97%)
Train Accuracy: 591/600 (98%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 600/10000 (98%)

Train teacher ID: 24
Train Accuracy: 582/600 (97%)
Train Accuracy: 591/600 (98%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 600/10000 (98%)

Train teacher ID: 25
Train Accuracy: 583/600 (97%)
Train Accuracy: 595/600 (99%)
Train Accuracy: 597/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accu

Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 600/10000 (98%)

Train teacher ID: 46
Train Accuracy: 591/600 (98%)
Train Accuracy: 596/600 (99%)
Train Accuracy: 597/600 (100%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 598/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 600/10000 (98%)

Train teacher ID: 47
Train Accuracy: 585/600 (98%)
Train Accuracy: 594/600 (99%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 600/10000 (98%)

Train teacher ID: 48
Train Accuracy: 590/600 (98%)
Train Accuracy: 596/600 (99%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accu

Train Accuracy: 598/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 599/600 (100%)
Test Accuracy: 599/10000 (98%)

Train teacher ID: 69
Train Accuracy: 586/600 (98%)
Train Accuracy: 596/600 (99%)
Train Accuracy: 596/600 (99%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 600/10000 (98%)

Train teacher ID: 70
Train Accuracy: 589/600 (98%)
Train Accuracy: 597/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 600/10000 (98%)

Train teacher ID: 71
Train Accuracy: 590/600 (98%)
Train Acc

Train Accuracy: 595/600 (99%)
Train Accuracy: 593/600 (99%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 600/10000 (98%)

Train teacher ID: 92
Train Accuracy: 587/600 (98%)
Train Accuracy: 597/600 (100%)
Train Accuracy: 597/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Test Accuracy: 600/10000 (98%)

Train teacher ID: 93
Train Accuracy: 586/600 (98%)
Train Accuracy: 595/600 (99%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 599/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 600/600 (100%)
Train Accuracy: 599/600 (100%)


## Train Student

In [8]:
train_student(model, dataset, nb_teachers)

pate.py:325: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output_softmax = F.softmax(output).data.numpy()


Computed Teacher 0 softmax predictions
Computed Teacher 1 softmax predictions
Computed Teacher 2 softmax predictions
Computed Teacher 3 softmax predictions
Computed Teacher 4 softmax predictions
Computed Teacher 5 softmax predictions
Computed Teacher 6 softmax predictions
Computed Teacher 7 softmax predictions
Computed Teacher 8 softmax predictions
Computed Teacher 9 softmax predictions
Computed Teacher 10 softmax predictions
Computed Teacher 11 softmax predictions
Computed Teacher 12 softmax predictions
Computed Teacher 13 softmax predictions
Computed Teacher 14 softmax predictions
Computed Teacher 15 softmax predictions
Computed Teacher 16 softmax predictions
Computed Teacher 17 softmax predictions
Computed Teacher 18 softmax predictions
Computed Teacher 19 softmax predictions
Computed Teacher 20 softmax predictions
Computed Teacher 21 softmax predictions
Computed Teacher 22 softmax predictions
Computed Teacher 23 softmax predictions
Computed Teacher 24 softmax predictions
Computed T

True